## 1. Pattern Detection

### 1.0 Load AST in memory (iterate over all ast in wordpress_source folder)

In [6]:
from code_analysis import AST, ASTReader
from pathlib import Path


# reader = ASTReader()
# ast = reader.read_ast("../example/example_1.php.ast.json")


## function visit node
'''
Iterate over each ast file (php.ast.json)
    call visitor
'''

### Get all the filenames in a given directory that sastifies a condition (*.ast.json)
def get_ast_json_files(directory):
   directory = Path(directory)
   return [str(p) for p in directory.rglob('*.ast.json')]

print(get_ast_json_files("../code_to_analyze/wordpress_ast"))
### Put all the filenames in an array
### Loop through array 

['../code_to_analyze/wordpress_ast/wp-mail.php.ast.json', '../code_to_analyze/wordpress_ast/wp-links-opml.php.ast.json', '../code_to_analyze/wordpress_ast/wp-trackback.php.ast.json', '../code_to_analyze/wordpress_ast/wp-settings.php.ast.json', '../code_to_analyze/wordpress_ast/wp-blog-header.php.ast.json', '../code_to_analyze/wordpress_ast/wp-comments-post.php.ast.json', '../code_to_analyze/wordpress_ast/wp-activate.php.ast.json', '../code_to_analyze/wordpress_ast/wp-config-sample.php.ast.json', '../code_to_analyze/wordpress_ast/readme.html.ast.json', '../code_to_analyze/wordpress_ast/wp-load.php.ast.json', '../code_to_analyze/wordpress_ast/index.php.ast.json', '../code_to_analyze/wordpress_ast/xmlrpc.php.ast.json', '../code_to_analyze/wordpress_ast/wp-login.php.ast.json', '../code_to_analyze/wordpress_ast/wp-cron.php.ast.json', '../code_to_analyze/wordpress_ast/wp-signup.php.ast.json', '../code_to_analyze/wordpress_ast/license.txt.ast.json', '../code_to_analyze/wordpress_ast/wp-admin/

### 1.1 Create a visitor than can select all the lines of code that calls a database

#### Example:

In PHP, there are multiple ways to achieve that:
```PHP
mysql_query ( * ) ;
mysqli_query ( * ) ;
$object - > execute () ;
$object - > mysql - > exec ( * ) ;
```

In [ ]:
# Code for loading

### 1.2 Parse AST with visitor to find all db calls

## 2. Known vulnerability detection

